交叉熵损失：一般用于分类问题 <br>
按照真实分布$p$来衡量识别一个样本所需的编码长度的期望，即平均编码长度（**信息熵**）: $H(p)=-\sum_{i=1}^Cp(x_i)log(p(x_i))$ <br>
使用拟合分布$q$来表示来自真实分布$p$的编码长度的期望，即平均编码长度（**交叉熵**）: $H(p,q)=-\sum_{i=1}^Cp(x_i)log(q(x_i))$ <br>
用交叉熵减去信息熵得到的是**相对熵**（KL散度）：$D(p||q)=H(p,q)-H(p)=\sum_{i=1}^Cp(x_i)log(\frac{p(x_i)}{q(x_i)})$ <br>
交叉熵可以看成是模型预测和标签之间的差距，减少模型预测与标签之间的差距即减小相对熵，由于一般信息熵不会变，所以只需要减小交叉熵（$H(p,q)\ge H(p)$）即可，所以选择交叉熵作为损失函数。

多分类问题的交叉熵损失函数：
$$
L = -\sum_{i=0}^{C-1}y_ilog(p_i)=-log(p_c)
$$
$p=[p_0,...p_{C-1}]$是一个概率分布，是模型预测的关于每个类的概率，$y=[y_0, ...,y_{C-1}]$是样本标签的one-hot表示，对应类别的位为1，其它为0（不进行标签平滑）

torch中在交叉熵中加入了softmax，相当于将nn.LogSoftmax()和nn.NLLLoss()组合在一个类中
$$
L = -log(\frac{e^{p_c}}{\sum_je^{p_j}})=-p_c+log(\sum_je^{p_j})
$$

In [7]:
import torch
import torch.nn as nn
import numpy as np
def CELoss(p ,y):
    p_c = torch.gather(p,dim=1 ,index=torch.argmax(y, dim=-1).unsqueeze(-1))
    exp_p_j = torch.exp(p)
    log_p = torch.log(torch.sum(exp_p_j, dim=-1))
    return torch.mean(-p_c + log_p)

y = torch.Tensor([[1,0,0], [0,1,0], [1,0,0]])
p = torch.from_numpy(
    np.array([
            [ 0.1545 , -0.5706, -0.0739 ],
            [ 0.2990, 0.1373, 0.0784],
            [ 0.1633, 0.0226, 0.1038 ]
    ])
)
loss_fn = nn.CrossEntropyLoss()
l1 = loss_fn(p, y)  # （预测值，标签）
l2 = CELoss(p,y)
print(l1)
print(l2)

tensor(0.9983, dtype=torch.float64)
tensor(0.9983, dtype=torch.float64)


torch.gather(p, dim, index)函数: p为待选取的张量，dim代表按什么方向选择，dim=0代表按列，=1代表按行，index表示每个dim方向选择哪一位的值。如果按行选择，index的行维度需要和p的行维度相等，按列选择，index的列维度需要和p的列维度相等。

In [13]:
import torch

p = torch.from_numpy(
    np.array([
            [ 0.1545, -0.5706, -0.0739 ],
            [ 0.2990, 0.1373, 0.0784],
            [ 0.1633, 0.0226, 0.1038 ]
    ])
)
print(torch.gather(p, dim=0, index=torch.Tensor([[1,0,2]]).long())) # 第一列第一行为0.2990，第二列第零行为-0.5706，第三列第二行为0.1038
print(torch.gather(p, dim=1, index=torch.Tensor([[1],[0],[2]]).long()))

tensor([[ 0.2990, -0.5706,  0.1038]], dtype=torch.float64)
tensor([[-0.5706],
        [ 0.2990],
        [ 0.1038]], dtype=torch.float64)
